In [1]:
import numpy as np
import matplotlib.pyplot as plt
from DEBT_control_class import DEBT_Control_Protocol
import logging, sys
logging.basicConfig(stream=sys.stderr, level=logging.ERROR)
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)


In [2]:
n = 6
params = dict()
params["custom"] = True
params["custom_routes"] = True
C = np.zeros((n, n))
for i in range(n-1):
    C[i, i+1] = 100
    C[n-1, 0] = 100
C = C + C.T
params["capacity_matrix"] = C

In [3]:
def create_path(i, j):
    path = np.zeros((n, n))
    k = i
    while k != j:
        path[k, (k+1) % n] = 1
        k = (k+1) % n
    return path

paths = dict()
for i in range(n):
    for j in range(i+1, n):
        paths[(i, j)] = [create_path(i, j), create_path(j, i)]
        paths[(j, i)] = paths[(i, j)].copy()
params["routes"] = paths

In [4]:
demand_params = dict()
demand_params["distribution"] = "Poisson"
demand_params["custom demands"] = True
mean = 10
sparsity = 0.5
demands = np.random.binomial(1, sparsity, (n, n))*np.random.poisson(mean, (n, n))
for i in range(n):
    demands[i,i] = 0
print("Demand Matrix:")
print(demands)
demand_params["demand_matrix"] = demands

Demand Matrix:
[[ 0 16  9 12  0  7]
 [ 0  0  7  0 10  6]
 [11  7  0  0  0  8]
 [ 8  8 11  0  0  6]
 [ 0 11  0 12  0  0]
 [ 0 12  0 14  0  0]]


In [5]:
price_params = dict()
price_params["stepsize"] = 0.01
price_params["threshold"] = 1*np.ones((n,n))
price_params["sensitivity"] = 0.1*np.ones((n,n)) # regularizer coefficient

In [6]:
# initialize the PCN and the protocol
myPCN = DEBT_Control_Protocol(params, demand_params, price_params) 
# loop for T time steps
T = 51

In [ ]:
# create arrays to store flows and prices
flows_data = dict()
path_price_data = dict()
for i in range(n):
    for j in range(n):
        if (i,j) not in myPCN.paths.keys():
            continue
        flows_data[(i,j)] = np.zeros((T,len(myPCN.paths[(i,j)])))
        path_price_data[(i,j)] = np.zeros((T,len(myPCN.paths[(i,j)])))
edge_price_data = np.zeros((T, n, n))
reset_data = np.zeros((T, n, n))

In [ ]:
logging.info("starting loop with %d iterations", T)
for t in range(T):
    logging.info("starting step %d", t)
    myPCN.single_step(t)
    edge_price_data[t] = myPCN.channel_prices.copy()
    for key in flows_data.keys():
        flows_data[key][t] = myPCN.flow_requests[key]
        path_price_data[key][t] = myPCN.path_prices[key]
        reset_data[t] = myPCN.channels_reset


In [ ]:
"""
fig = plt.figure(figsize = (7,4))
# gs = fig.add_gridspec(2, hspace=0)
axs = fig.subplots(2, sharex=True)
# plt.subplots_adjust(hspace=0)
i = 0
j = 1
axs[0].plot(flows_data[(i,j)][:,0], label = "short\npath", marker ='.')
axs[0].plot(flows_data[(i,j)][:,1], label = "long\npath", marker ='.')
axs[1].plot(path_price_data[(i,j)][:,0], label = "short\npath", marker ='.')
axs[1].plot(path_price_data[(i,j)][:,1], label = "long\npath", marker ='.')
plt.legend(bbox_to_anchor=(0.95, 1.4), loc="upper left")
plt.xticks(range(0, T+1, 3))
axs[0].set(yticks = range(0, 11, 5))
axs[1].set(yticks = [-0.2, 0, 0.2])
axs[0].set(ylabel='Flow')
axs[1].set(xlabel = 'Time', ylabel='Price')
# axs[0].grid(True, axis='x')
# axs[1].grid(True, axis='x')
# fig.tight_layout()
fig.savefig("cyclic_example_smooth.pdf")
"""

In [ ]:
fig = plt.figure(figsize = (7,4))
# gs = fig.add_gridspec(2, hspace=0)
axs = fig.subplots(2, sharex=True)
# plt.subplots_adjust(hspace=0)
i = 0
j = 2
axs[0].plot(flows_data[(i,j)][:,0], label = "A -> C", marker ='.')
axs[0].plot(flows_data[(i+1,j)][:,0], label = "B -> C", marker ='.')
axs[1].plot(path_price_data[(i,j)][:,0], label = "A -> C", marker ='.')
axs[1].plot(path_price_data[(i+1,j)][:,0], label = "B -> C", marker ='.')
plt.legend(bbox_to_anchor=(0, 1.3), loc="upper left")
plt.xticks(range(0, 51, 5))
axs[0].set(yticks = range(0, 11, 5))
axs[1].set(yticks = [0, 0.5, 1])
axs[0].set(ylabel='Flow')
axs[1].set(xlabel = 'Time', ylabel='Price')
# axs[0].grid(True, axis='x')
# axs[1].grid(True, axis='x')
# fig.tight_layout()
fig.savefig("deadlock_example_smooth.pdf")

In [ ]:
alphabets = ['A', 'B', 'C', 'D', 'E', 'F']
plt.close("all")
plt.figure(figsize=(6,4))
for i in range(n):
    for j in range(n):
        if myPCN.demands[i,j] > 0:
            for k in range(len(myPCN.paths[(i,j)])):
                plt.plot(flows_data[(i,j)][:,k], label = alphabets[i] + " -> " + alphabets[j] + ", path " + str(k))
plt.legend()
plt.title("Flows as a function of time")

In [ ]:
plt.figure(figsize=(6,4))
for i in range(n):
    for j in range(n):
        if myPCN.demands[i,j] > 0:
            for k in range(len(myPCN.paths[(i,j)])):
                plt.plot(path_price_data[(i,j)][:,k], label = alphabets[i] + " -> " + alphabets[j] + ", path " + str(k))
plt.legend()
plt.title("Prices as a function of time")